# Clustering
#### Part of the course on "Foundations of machine learning", Department of Mathematics and Statistics, University of Turku, Finland
#### Lectures available on YouTube: https://youtube.com/playlist?list=PLbkSohdmxoVAZ9DEHEWHjeGK7Ei-DjKHI&si=Msu74_I0qhLrRWcu
#### Code available on GitHub: https://github.com/ionpetre/FoundML_course_assignments

#### This notebook is based on the following sources: 

> 

A Multilayer Perceptron (MLP) is a type of feedforward artificial neural network that consists of multiple layers of interconnected artificial neurons, or nodes. They can be used for both classification, as well as regression problems. We demonstrate them in this notebook in their simplest format, with a single hidden layer, in addition to the input and the output layers. 

In this noteebook we use the MNIST dataset to train an MLP classifier, and a World Health Organisation dataset on life expectancy in various countries to train an MLP regressor. 

#### Load the libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, average_precision_score, roc_auc_score

In [ ]:
# Reset the seed of the random number generator, for reproducibility purposes

import os

def reset_seed(SEED = 0):
    """Reset the seed for every random library in use (System, numpy)"""

    os.environ['PYTHONHASHSEED']=str(SEED)
    np.random.seed(SEED)


reset_seed(2023)

## I. Demo MLP classifiers on the MNIST dataset

#### The MNIST dataset: 

The MNIST dataset is a widely used dataset in the field of machine learning and computer vision. It consists of a collection of handwritten digits, and its name stands for the "Modified National Institute of Standards and Technology" dataset. 

The MNIST dataset contains a total of 70,000 grayscale images of the 0-9 handwritten digits. The images are 28x28 pixels. The MNIST dataset consists of a training set of 60,000 images and a test set of 10,000 images. The MNIST dataset is commonly used for tasks related to digit recognition, classification, and image analysis.
It serves as a benchmark dataset for developing and evaluating machine learning algorithms and deep learning models.

While MNIST is a well-known dataset, it is relatively simple compared to some real-world problems. Other datasets with more complex images exist, such as CIFAR-10, ImageNet, and others.

In [ ]:
from keras.datasets import mnist
from keras.utils import to_categorical

img_rows, img_cols = 28, 28
num_classes = 10

(X_train, y_train), (X_test, y_test) = mnist.load_data()

print('We have %2d training pictures and %2d test pictures.' % (X_train.shape[0],X_test.shape[0]))
print('Each picture is of size (%2d,%2d)' % (X_train.shape[1], X_train.shape[2]))

In [ ]:
# Display some images

def display_train_image(position):
    plt.figure(figsize=(1,1))
    plt.title('Example %d. Label: %d' % (position, y_train[position]))
    plt.imshow(X_train[position], cmap=plt.cm.gray_r)
    plt.show()
    plt.close()
    
for i in range(50):
    display_train_image(1200*i)

In [ ]:
# Is the dataset balanced?

y_train_count = np.unique(y_train, return_counts=True)
df_y_train = pd.DataFrame({'Label':y_train_count[0], 'Count':y_train_count[1]})
df_y_train

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, 
    y_train, 
    test_size=0.2, 
    random_state=150, 
    stratify=y_train,
    shuffle=True
)

# Check the result of the data split

print('# of training images:', X_train.shape[0])
y_train_count = np.unique(y_train, return_counts=True)
df_y_train = pd.DataFrame({'Label':y_train_count[0], 'Train samples':y_train_count[1]})
print(df_y_train.to_string(index=False))

print('# of validation images:', X_valid.shape[0])
y_valid_count = np.unique(y_valid, return_counts=True)
df_y_valid = pd.DataFrame({'Label':y_valid_count[0], 'Valid samples':y_valid_count[1]})
print(df_y_valid.to_string(index=False))

In [ ]:
# Standardise the data

# Reshape the data so that each 28 x 28 picture is transformed into a 784-long vector
X_train=X_train.reshape(X_train.shape[0],-1)
print("Shape of the training data: ",X_train.shape)
X_valid=X_valid.reshape(X_valid.shape[0],-1)
X_test=X_test.reshape(X_test.shape[0],-1)


from sklearn.preprocessing import MinMaxScaler

minmax_scaler = MinMaxScaler()
minmax_scaler.fit(X_train)

X_train_std = minmax_scaler.transform(X_train)
X_valid_std = minmax_scaler.transform(X_valid)
X_test_std  = minmax_scaler.transform(X_test)

#### Train a multilayer perceptron classifier on the MNIST dataset

In [ ]:
from sklearn.neural_network import MLPClassifier

MLPclf = MLPClassifier(
    hidden_layer_sizes=(10,), # number of neurons in the hidden layer
    activation='logistic', 
    random_state=150, 
    max_iter=300,       # number of epochs   
    batch_size=500,     # batch size for mini-batch training 
    learning_rate='constant', 
    learning_rate_init=0.001, 
    shuffle=True,
    verbose=True,
    early_stopping=False, 
    n_iter_no_change=10,
    tol=1e-4,
    solver='sgd',       # stochastic gradient descent
)

MLPclf.fit(X_train, y_train)

In [ ]:
print("Number of parameters in the model:", MLPclf.coefs_[0].size+MLPclf.coefs_[1].size)


#### Check the quality of the predictions through the accuracy score (on train and validation) and through the confusion matrix.

In [ ]:
y_train_pred = MLPclf.predict(X_train)
y_valid_pred = MLPclf.predict(X_valid)

print("The classification results on the training data:")
print(classification_report(y_train,y_train_pred))
print("Confusion matrix (training data):\n", confusion_matrix(y_train,y_train_pred))

print("\n The classification results on the validation data:")
print(classification_report(y_valid,y_valid_pred))
print("Confusion matrix (validation data):\n", confusion_matrix(y_valid,y_valid_pred))

#### Check the learning curve per epoch

In [ ]:
train_loss_10_500_log = MLPclf.loss_curve_

plt.plot(train_loss_10_500_log, color='green', alpha=0.8, label='10 hidden neurons, batch=500, logistic activation')
plt.title("Loss over epochs", fontsize=14)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()

#### Try a few other setups

In [ ]:
MLPclf = MLPClassifier(
    hidden_layer_sizes=(20,), # number of neurons in the hidden layer
    activation='logistic', 
    random_state=150, 
    max_iter=300,       # number of epochs   
    batch_size=300,     # batch size for mini-batch training 
    learning_rate='constant', 
    learning_rate_init=0.001, 
    shuffle=True,
    verbose=True,
    early_stopping=False, 
    n_iter_no_change=10,
    tol=1e-4,
    solver='sgd',       # stochastic gradient descent
)

MLPclf.fit(X_train, y_train)
train_loss_20_300_log = MLPclf.loss_curve_


MLPclf = MLPClassifier(
    hidden_layer_sizes=(50,), # number of neurons in the hidden layer
    activation='logistic', 
    random_state=150, 
    max_iter=300,       # number of epochs   
    batch_size=1000,     # batch size for mini-batch training 
    learning_rate='constant', 
    learning_rate_init=0.001, 
    shuffle=True,
    verbose=True,
    early_stopping=False, 
    n_iter_no_change=10,
    tol=1e-4,
    solver='sgd',       # stochastic gradient descent
)

MLPclf.fit(X_train, y_train)
train_loss_50_1000_log = MLPclf.loss_curve_


MLPclf = MLPClassifier(
    hidden_layer_sizes=(30,), # number of neurons in the hidden layer
    activation='relu', 
    random_state=150, 
    max_iter=300,       # number of epochs   
    batch_size=300,     # batch size for mini-batch training 
    learning_rate='constant', 
    learning_rate_init=0.001, 
    shuffle=True,
    verbose=True,
    early_stopping=False, 
    n_iter_no_change=10,
    tol=1e-4,
    solver='sgd',       # stochastic gradient descent
)

MLPclf.fit(X_train, y_train)
train_loss_30_300_relu = MLPclf.loss_curve_


MLPclf = MLPClassifier(
    hidden_layer_sizes=(20,), # number of neurons in the hidden layer
    activation='tanh', 
    random_state=150, 
    max_iter=300,       # number of epochs   
    batch_size=300,     # batch size for mini-batch training 
    learning_rate='constant', 
    learning_rate_init=0.001, 
    shuffle=True,
    verbose=True,
    early_stopping=False, 
    n_iter_no_change=10,
    tol=1e-4,
    solver='sgd',       # stochastic gradient descent
)

MLPclf.fit(X_train, y_train)
train_loss_20_300_tanh = MLPclf.loss_curve_


In [ ]:
plt.figure(figsize=(10,5))

plt.plot(train_loss_10_500_log, color='green', alpha=0.8, label='10 hidden neurons, batch=500, logistic activation')
plt.plot(train_loss_20_300_log, color='red', alpha=0.8, label='20 hidden neurons, batch=300, logistic activation')
plt.plot(train_loss_50_1000_log, color='blue', alpha=0.8, label='50 hidden neurons, batch=1000, logistic activation')
plt.plot(train_loss_30_300_relu, color='orange', alpha=0.8, label='30 hidden neurons, batch=300, relu activation')
plt.plot(train_loss_20_300_tanh, color='brown', alpha=0.8, label='20 hidden neurons, batch=300, tanh activation')
plt.title("Loss over epochs", fontsize=14)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()

#### Conclusion
Different parameter setups can have quite a drastic effect on the performance of the MLP model!

#### Train an MLP with different combinations of parameters to find better setups. You can add more options to the combination.
#### NOTE: each combination of parameters will get a new fit. Expect about one minute or so per fit on a "standard" computer.

In [ ]:
param_grid = {
              "hidden_layer_sizes": [(10,), (20,)],
              "activation": ['logistic', 'relu', 'tanh'],
              "batch_size": [200, 500],
              }

MLPclf = MLPClassifier(
    hidden_layer_sizes=(20,), #*
    activation='logistic', #*
    random_state=150, 
    max_iter=300,          
    batch_size=200,      #*
    learning_rate='constant', 
    learning_rate_init=0.001, 
    shuffle=True,
    verbose=True,
    early_stopping=False, 
    n_iter_no_change=10,
    tol=1e-4,
    solver='sgd', 
)

from sklearn.model_selection import GridSearchCV

gcv = GridSearchCV(
    estimator = MLPclf,
    cv = 3,
    param_grid = param_grid,
    refit = True,
    n_jobs = -1,
    verbose = 1,
)
gcv.fit(X_train,y_train)

pd.DataFrame(gcv.cv_results_)

In [ ]:
best_MLPclf = gcv.best_estimator_

print("Number of parameters in the model:", best_MLPclf.coefs_[0].size + best_MLPclf.coefs_[1].size)

y_train_pred = best_MLPclf.predict(X_train)
y_valid_pred = best_MLPclf.predict(X_valid)
y_test_pred = best_MLPclf.predict(X_test)

print("The classification results on the training data:")
print(classification_report(y_train,y_train_pred))
print("Confusion matrix (training data):\n", confusion_matrix(y_train,y_train_pred))

print("\n The classification results on the validation data:")
print(classification_report(y_valid,y_valid_pred))
print("Confusion matrix (validation data):\n", confusion_matrix(y_valid,y_valid_pred))

print("\n The classification results on the test data:")
print(classification_report(y_test,y_test_pred))
print("Confusion matrix (test data):\n", confusion_matrix(y_test,y_test_pred))

#### Conclusion
The MLP model got an accurracy of 0.93 on the test dataset, with very high precision and recall across all classes. It does suffer a little from the class imbalance: the perfoormance is slightly better for the larger classes. 

In [ ]:
del X_train
del X_valid
del X_test
del y_train
del y_valid
del y_test
del MLPclf
del best_MLPclf

## Challenge: Life expectancy model

We will train an MLP regression model to predict the life expectancy in a country, depending on a set of key factors in that country: infantile deaths, alcohol consumption, vaccination data, GDP, body mass index, schooling, and others. Some definitions of these factors can be found at https://datahelpdesk.worldbank.org/knowledgebase/topics/19286-world-development-indicators-wdi. 

The health data was collected 2000-2015 in 193 countries and made available under the Global Health Observatory (GHO) data repository of the World Health Organization (WHO). The economic data was collected from United Nation website. Several data files have been merged together into a single dataset. The dataset aims to help answer the following key questions:

>Does the life expectancy reliably depend on the factors included in this dataset? 

>How much should a country having a lower life expectancy value (<65) increase its healthcare expenditure in order to improve its average lifespan?

>Does life expectancy has positive or negative correlation with eating habits, lifestyle, exercise, smoking, drinking alcohol etc.

>What is the impact of schooling on the lifespan of humans?

>What is the impact of Immunization coverage on life expectancy?

Data source: https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who (author: KUMARRAJARSHI). The data was collected from WHO and United Nations website with the help of Deeksha Russell and Duan Wang.


#### Load the dataset
For this challenge, you need to download the training and the test datasets from Moodle (or from the Kaggle source above) and make sure it is saved in the same folder as your code or indicate the relative folder location in the read function below. 

In [ ]:
X = pd.read_csv("2015_WHO_Life_Expectancy_Data.csv")

#### Q1. How many features you have in the dataset? 
#### Q2. How many data points do you have in the dataset? 
#### Q3. Do you have missing values in the dataset? 

In [ ]:
# Your code here


#### Check the categorical features

In [ ]:
# find the categorical features

categorical = [var for var in X.columns if X[var].dtype=='O']
print('There are {} categorical variables\n'.format(len(categorical)))
print('The categorical variables are :', categorical)

# check for missing values in categorical variables 

X[categorical].isnull().sum()

In [ ]:
print(X['Status'].value_counts())
print(X['Country'].value_counts())

#### We will leave the categorical features encoded as they are, they will not be used in training the model.

#### Check the numerical features

In [ ]:
# Find the numerical variables

numerical = [var for var in X.columns if X[var].dtype!='O']
print('There are {} numerical variables\n'.format(len(numerical)))
print('The numerical variables are :', numerical)

# check missing values in numerical variables
X[numerical].isnull().sum()

In [ ]:
# Try to remove all datapoints with at least one of the features missing

X_complete = X.dropna(axis=0)
print(X_complete.info())

# We conclude that we lose about half of the data, on a dataset that is relatively small to start with.
# We give up on this idea

del X_complete

In [ ]:
# Check some statistical indicators of the numerical features to decide how to handle the missing values

X[numerical].describe()

#### Data curation decisions
1. 10 values missing from life expectancy, which will be our target values. Drop the rows missing the values.
2. The minimal values in all columns are 0 or very close to 0. This is unrealistic and it probably indicates empty entries encoded with such small number. This suggests that replacing empty values with 0 is in line with the way the data is already filled in. This is a big decision that in a more careful modeling scenario may have to be tested against other imputation techniques. 

In [ ]:
print(X.info())

In [ ]:
# Drop the rows with empty life expectancy
X = X[X['Life expectancy '].notna()]

# Fill in the remaining empty values with 0
X = X.fillna(0)

print(X.info())
X.describe()

In [ ]:
# Extract the target variable from the dataset

y = X['Life expectancy ']
X = X.drop(['Life expectancy '], axis=1)

# Update the list of the numerical variables
numerical.remove("Life expectancy ")

#### Design decisions
>One question is whether the split should be stratified or not, and if yes, based on what column. One possibility would be to stratify it based on the developed/under development status of each country, except that the values in those columns seem strange (check for example the status for Finland). In this notebook we do an un-stratified split. 

>Another question is whether the split should be grouped, i.e., all data points from a single country to be placed together either in train or in validation. This is often done in medical setups, for example on patient data, to avoid data leakage in-between train and validation, due to correlations between the samples from the same patient. In this notebook we will not do a grouped split, but this may be an option to be tested in a more extensive analysis. 

>Another decision is whether to do a traditional train/validation/test design, or whether to run a cross-validation setup. The relatively small dataset indicates that a CV may be a good idea. For simplicity though, we implement a traditional train/validation/test setup here, but this may also be changed in a more extensive analysis. 

>We will have to standardize the (numerical features of the) data. We use StandardScaler to bring them to mean 0 and standard deviation 1. 

>The training will be done only on the numerical features. 

In [ ]:
# Split the data into train/validation/test

X_train_valid, X_test, y_train_valid, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=120, 
    shuffle=True
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_valid, 
    y_train_valid, 
    test_size=0.25, 
    random_state=120, 
    shuffle=True
)

del X_train_valid
del y_train_valid

# convert to pandas dataframe
X_train = pd.DataFrame(X_train, columns=X.columns)
X_valid = pd.DataFrame(X_valid, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)
y_train = pd.DataFrame(y_train)
y_valid = pd.DataFrame(y_valid)
y_test = pd.DataFrame(y_test)

#### Feature scaling
We use StandardScaler to get all numerical features with mean 0 and std 1.

In [ ]:
# Your code here



#### Model training
We train an MLP on the numerical features of X_train to predict the life expectancy variable. 

In [ ]:
# Apply the same scaler to the validation data. 
# Your code here



In [ ]:
# Train an MLP regression model

from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score as R2
from sklearn.neural_network import MLPRegressor

MLPreg = MLPRegressor(
    hidden_layer_sizes=(20,), 
    activation='logistic', 
    random_state=150, 
    max_iter=1000,          
    batch_size=20,      
    learning_rate='constant', 
    learning_rate_init=0.001, 
    shuffle=True,
    verbose=True,
    early_stopping=False, 
    n_iter_no_change=10,
    tol=1e-4,
    solver='sgd', 
)

# Your code here









plt.figure(figsize=(10,3))
plt.plot(MLPreg.loss_curve_, color='green', alpha=0.8, label='20 hidden neurons, batch=20, logistic activation')
plt.title("Loss over epochs", fontsize=14)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()

#### Q4. How many parameters does your model have? 
#### Q5. What is the mean absolute error on the validation data (2 decimals)? 

The R2 score indicates a great result, but the number of parameters is potentially large for the relatively small dataset we have. Let's train an MLP regressor with 60 parameters. Keep the batch size the same (for reproducibility purposes). This will be the smallest model with R2 on validation at least 0.90 (mind the rounding up to 2 decimals). 

In [ ]:
# Train a smaller MLP regression model

# Your code here










plt.figure(figsize=(10,3))
plt.plot(smallMLPreg.loss_curve_, color='green', alpha=0.8, label='MLP regressor for the life expectancy model')
plt.title("Loss over epochs", fontsize=14)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()

#### Q6. How many hidden neurons does the smaller model have? 
#### Q7. What is the MAE score of the smaller model on the validation data (2 decimals only)? 

In [ ]:
# Check the smaller model on the test data. Remember the standardisation of the data. 
# Your code here



#### Q8. What is the MAE score of the smaller model on the test data (2 decimals only)? 

#### Sample code on how to use the model. 

In [ ]:
# Check the prediction on Finland for year 2015.

my_country = "Finland"
my_year = 2015

filter = (X['Country'] == my_country) & (X['Year']==my_year)

my_datapoint = X[filter]
y_true = y[X.index[filter]]

print("My data:")
#display(my_datapoint)

scaled_X = pd.DataFrame(scaler.transform(my_datapoint[numerical]), columns=X[numerical].columns)
y_pred = smallMLPreg.predict(scaled_X)

df = pd.DataFrame(
    {'Country': X[filter]['Country'],
     'Year': X[filter]['Year'],
     'True life exp':y_true, 
     'Predicted life exp': np.around(y_pred, decimals=1)
    }
)

display(df)

In [ ]:
# Check the prediction on Finland over several years.

my_country = "Finland"
my_year = 2015

filter = (X['Country'] == my_country) 

my_datapoint = X[filter]
y_true = y[X.index[filter]]

print("My data:")
#display(my_datapoint)

scaled_X = pd.DataFrame(scaler.transform(my_datapoint[numerical]), columns=X[numerical].columns)
y_pred = smallMLPreg.predict(scaled_X)

df = pd.DataFrame(
    {'Country': X[filter]['Country'],
     'Year': X[filter]['Year'],
     'True life exp':y_true, 
     'Predicted life exp': np.around(y_pred, decimals=1)
    }
)

#display(df)

df.set_index('Year')[['True life exp', 'Predicted life exp']].plot()